In [21]:
import os

import google.oauth2.credentials

from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google_auth_oauthlib.flow import InstalledAppFlow

# The CLIENT_SECRETS_FILE variable specifies the name of a file that contains
# the OAuth 2.0 information for this application, including its client_id and
# client_secret.
CLIENT_SECRETS_FILE = "client_secret.json"

# This OAuth 2.0 access scope allows for full read/write access to the
# authenticated user's account and requires requests to use an SSL connection.
SCOPES = ['https://www.googleapis.com/auth/youtube.force-ssl']
API_SERVICE_NAME = 'youtube'
API_VERSION = 'v3'

def get_authenticated_service():
  flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS_FILE, SCOPES)
  credentials = flow.run_console()
  return build(API_SERVICE_NAME, API_VERSION, credentials = credentials)

def channels_list_by_username(service, **kwargs):
  results = service.channels().list(
    **kwargs
  ).execute()
  
  print('This channel\'s ID is %s. Its title is %s, and it has %s views.' %
       (results['items'][0]['id'],
        results['items'][0]['snippet']['title'],
        results['items'][0]['statistics']['viewCount']))

def channels_by_channel_id(service, **kwargs):
  results = service.channels().list(
    **kwargs
  ).execute()
  
  print(results['items'][0]['id'],
        results['items'][0]['snippet']['title'],
        results['items'][0]['statistics']['viewCount'],
        results['items'][0]['statistics']['commentCount'],
        results['items'][0]['statistics']['subscriberCount'],
        results['items'][0]['statistics']['videoCount'])

if __name__ == '__main__':
  # When running locally, disable OAuthlib's HTTPs verification. When
  # running in production *do not* leave this option enabled.
  os.environ['OAUTHLIB_INSECURE_TRANSPORT'] = '1'
  service = get_authenticated_service()

  channels_list_by_username(service,
      part='snippet,contentDetails,statistics',
      forUsername='GoogleDevelopers')
  channels_by_channel_id(service,
      part='snippet,contentDetails,statistics',
      id='UC_x5XG1OV2P6uZZ5FSM9Ttw')


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=56613175135-nb6ihl7fn6s29s4bf02ujteuflr9r6rk.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.force-ssl&state=S8Hg7nKv9h8PQ94pNa8S79XMYQWsjg&prompt=consent&access_type=offline
Enter the authorization code: 4/AACsaNSrf3VTdBH9swstXoRGzlcwgqz7fGzx4xlwM7BM5Od5gPV926Y
This channel's ID is UC_x5XG1OV2P6uZZ5FSM9Ttw. Its title is Google Developers, and it has 129296688 views.
UC_x5XG1OV2P6uZZ5FSM9Ttw Google Developers 129296688 393 1346631 4598


In [25]:
import pandas as pd
import os

import google.oauth2.credentials
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google_auth_oauthlib.flow import InstalledAppFlow

# The CLIENT_SECRETS_FILE variable specifies the name of a file that contains
# the OAuth 2.0 information for this application, including its client_id and
# client_secret.
CLIENT_SECRETS_FILE = "client_secret.json"

# This OAuth 2.0 access scope allows for full read/write access to the
# authenticated user's account and requires requests to use an SSL connection.
SCOPES = ['https://www.googleapis.com/auth/youtube.force-ssl']
API_SERVICE_NAME = 'youtube'
API_VERSION = 'v3'

def get_authenticated_service():
  flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS_FILE, SCOPES)
  credentials = flow.run_console()
  return build(API_SERVICE_NAME, API_VERSION, credentials = credentials)

def get_channel_information(service, **kwargs):
  results = service.channels().list(
    **kwargs
  ).execute()
  
  return (results['items'][0]['id'],
        results['items'][0]['snippet']['title'],
        results['items'][0]['statistics']['viewCount'],
        results['items'][0]['statistics']['commentCount'],
        results['items'][0]['statistics']['subscriberCount'],
        results['items'][0]['statistics']['videoCount'])

df = pd.read_csv('influencer_list.csv')

columns = ['tw_handle','yt_url','yt_title', 'yt_channel_id','yt_user_id','yt_subscriber_count',
           'yt_view_count','yt_video_count']
outdf = pd.DataFrame(columns=columns)

def get_last_part(url):
    splits = url.split('/')
    if url:
        if url.endswith('/'):
            last_part = splits[-2]
        else:
            last_part = splits[-1]
    else:
        last_part = None    
    return last_part

for index, row in df.iterrows():
    twitter_url = row['Twitter']
    tw_handle = get_last_part(twitter_url)
    youtube_url = row['Youtube']
    yt_url = youtube_url
    if 'channel' in youtube_url:
        yt_channel_id = get_last_part(yt_url)
        result = get_channel_information(service,
          part='snippet,contentDetails,statistics',
          id=yt_channel_id)
    elif 'user' in youtube_url:
        yt_user_id = get_last_part(yt_url)
        result = get_channel_information(service,
          part='snippet,contentDetails,statistics',
          forUsername=yt_user_id)
    else:
        yt_channel_id = None
        yt_user_id = None
    
    yt_channel_id = result[0]
    yt_title = result[1]
    yt_view_count = result[2]
    yt_comment_count = result[3]
    yt_subscriber_count = result[4]
    yt_video_count = result[5]
    
    result_list = [tw_handle, yt_url, yt_title, yt_channel_id, yt_user_id, yt_subscriber_count, 
                   yt_view_count, yt_video_count]
    outdf.loc[index] = result_list

outdf.to_csv('youtube_influencers_details.csv', sep=';', index=False)

In [26]:
outdf.shape

(90, 8)

In [42]:
from klout import Klout

api_key = 'qzcujyjgy7b554kw9htud9ny'
k = Klout(api_key, secure=True)

def get_handle(url):
    splits = url.split('/')
    if url:
        if url.endswith('/'):
            handle = splits[-2]
        else:
            handle = splits[-1]
    else:
        handle = None    
    return handle

def get_klout_id(screen_name):
    kloutId = k.identity.klout(screenName=screen_name).get('id')
    return kloutId

def get_klout_score(kloutId):
    score = k.user.score(kloutId=kloutId, timeout=5).get('score')
    return score

df = pd.read_csv('twitter_influencers_details.csv',sep=';')

columns = ['tw_handle', 'tw_screen_name', 'kl_id,', 'kl_score']
outdf = pd.DataFrame(columns=columns)

for index, row in df.iterrows():
    tw_handle = row['tw_handle']
    tw_screen_name = row['tw_screen_name']
    kloutId = get_klout_id(tw_screen_name)
    score = get_klout_score(kloutId)
    
    result_list = [tw_handle, tw_screen_name, kloutId, score]
    outdf.loc[index] = result_list

outdf.to_csv('klout_influencers_details.csv', sep=';', index=False)

In [31]:
api_key = 'qzcujyjgy7b554kw9htud9ny'
k = Klout(api_key, secure=True)
# k.user.influence(kloutId='11747')
score = k.user.score(kloutId='11747', timeout=5).get('score')
score

45.2625715724686